In [2]:
import tensorflow as tf

import matplotlib.pyplot as plt

# Custom Loss Function

In [35]:
# Define the model with 1 neuron
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1, input_shape=(1, ))
])

## Using Loss Function

We typically define the loss function when compiling our model. This can be achieved either by using a string representing its name or by directly passing the loss object.

In [4]:
# Compile with loss using string representation of its name
model.compile(loss="mean_squared_error", optimizer="sgd")

# Compile with loss using the loss object
model.compile(loss=tf.keras.losses.MeanSquaredError, optimizer="sgd")

Both approaches are valid and will produce the same result.

The significant advantage of using the loss object is the ability to include additional parameters in its call, which offers more customization options.

In [5]:
# Compile with loss object and additional parameters
model.compile(loss=tf.keras.losses.MeanSquaredError(reduction="sum"), optimizer="sgd")

In this example, we pass an additional parameter delta to the mean_squared_error loss function, allowing for more fine-tuned control over the loss calculation. This flexibility is not available when using the loss function name as a string.

## Example: Huber Loss

The Huber Loss function is a type of loss function used in regression tasks. It is less sensitive to outliers compared to the Mean Squared Error (MSE) loss function. The Huber Loss function combines the best properties of the Mean Absolute Error (MAE) and the Mean Squared Error (MSE) loss functions.

The formula for the Huber Loss function is defined as:


$$L_{\delta}(y, \hat{y}) = \begin{cases}
\frac{1}{2}(y - \hat{y})^2, & \text{if } |y - \hat{y}| \leq \delta \\
\delta(|y - \hat{y}| - \frac{1}{2}\delta), & \text{otherwise}
\end{cases}$$

Where:
- $y$ represents the true target value.
- $\hat{y}$ represents the predicted value by the model.
- $\delta$ (delta) is a threshold parameter that determines the point at which the loss function transitions from quadratic to linear.

The Huber Loss function penalizes the model predictions quadratically when the absolute error (difference between the true and predicted values) is small (i.e., within the range of $\delta$). Beyond this range, the loss becomes linearly increasing with the absolute error, providing robustness to outliers.

## Creating a Custom Loss Function

In [6]:
def custom_loss_function(y_true, y_pred):
    """Custom loss function for evaluating the performance of a neural network model.

    This function calculates the loss value based on a custom formula or logic defined
    within it. It takes two arguments, `y_true` and `y_pred`, representing the true labels
    or ground truth values and the predicted values generated by the model, respectively.

    Parameters:
    - y_true (tensor): True labels or ground truth values.
    - y_pred (tensor): Predicted values generated by the model.

    Returns:
    - loss (tensor): Calculated loss value.

    Example usage:
    ```python
    model.compile(loss=custom_loss_function, optimizer='adam')
    ```
    """
    loss = ...
    return loss

In [60]:
def get_huber_loss(delta=1.0):
    def _huber_loss(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= delta
        small_error_loss = tf.square(error) / 2
        large_error_loss = delta * (tf.abs(error) - delta / 2)
        return tf.where(is_small_error, small_error_loss, large_error_loss)

    return _huber_loss

In [61]:
# Compile with custom loss function
model.compile(loss=get_huber_loss(), optimizer="sgd")

## Creating a Custom Loss Function as Class Object

In [78]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, delta=1.0):
        """Initialize the Huber loss with a specified delta.

        Parameters:
        - delta (float): Threshold parameter for the Huber loss function.
        """
        super().__init__()
        self.delta = delta

    def __call__(self, y_true, y_pred, sample_weight=None):
        """Calculate the Huber loss.

        Parameters:
        - y_true (tensor): True labels or ground truth values.
        - y_pred (tensor): Predicted values generated by the model.

        Returns:
        - loss (tensor): Calculated Huber loss value.
        """
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= self.delta
        small_error_loss = tf.square(error) / 2
        large_error_loss = self.delta * (tf.abs(error) - self.delta / 2)
        return tf.where(is_small_error, small_error_loss, large_error_loss)

In [79]:
# Compile with custom loss function as class object
model.compile(loss=HuberLoss(delta=.1), optimizer="sgd")